### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)


In [2]:
#show ehead of table
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#show unique value foe Gender
purchase_data["Gender"].unique()

array(['Male', 'Other / Non-Disclosed', 'Female'], dtype=object)

## Player Count

* Display the total number of players


In [4]:
#Total # of players
total_number_player=len(list(purchase_data["SN"].unique()))
total_number_player

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
number_unique_item=len(purchase_data["Item ID"].unique())
#number_unique_item
average_price =purchase_data["Price"].mean()
#average_price
number_purchase=purchase_data['Purchase ID'].count()
#number_purchase
total_revenue=purchase_data['Price'].sum()
#total_revenue
purchasing_analyses=pd.DataFrame({"unique Items":[number_unique_item],
                                  "Average Price":[average_price],
                                  "Number_Purchase":[number_purchase],
                                  "Total Revenue":[total_revenue]
                                 })
purchasing_analyses

,unique Items,Average Price,Number_Purchase,Total Revenue
0,183,3.050987,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
# small data frame that needed for gender analyese ,2columns
gender_data=purchase_data.loc[:,["SN","Gender"]]
gender_data.head()

,SN,Gender
0,Lisim78,Male
1,Lisovynya38,Male
2,Ithergue48,Male
3,Chamassasya86,Male
4,Iskosia90,Male


In [7]:
#Remove Duplicate 
refined_gender=gender_data.drop_duplicates(subset=["SN","Gender"],keep='first',inplace=False)
#now , calculate demographics
demo_gender=refined_gender.groupby("Gender").count()
demo_gender["percent"]=demo_gender["SN"]/total_number_player*100
#rename columns
demo_gender.columns=["Total_count","Percent"]
demo_gender


,Total_count,Percent
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722


In [8]:
# another way,
gender_data=purchase_data.loc[:,["SN","Gender"]]
male_unic_player=gender_data[gender_data["Gender"]=="Male"].groupby(["SN"]).count()
count_male=int(male_unic_player.count())

female_unic_player=gender_data[gender_data["Gender"]=="Female"].groupby(["SN"]).count()
count_female=int(female_unic_player.count())

other_unic_player=gender_data[gender_data["Gender"]=="Other / Non-Disclosed"].groupby(["SN"]).count()
count_other=int(other_unic_player.count())

male_percent=int(count_male/total_number_player*10000)/100
female_percent=int(count_female/total_number_player*10000)/100
other_percent =int(count_other/total_number_player*10000)/100

gender_demographics=pd.DataFrame({"Gender":["Male","Female","Other / Non-Disclosed"],
                                  "Total_Count":[count_male,count_female,count_other],
                                 "Percent":[male_percent,female_percent,other_percent]})

gender_demographics

,Gender,Total_Count,Percent
0,Male,484,84.02
1,Female,81,14.06
2,Other / Non-Disclosed,11,1.90



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
#calculate
purchase_count_gender=purchase_data.groupby(["Gender"])["Item ID"].count()
ave_purchase_price_gender=purchase_data.groupby(["Gender"])["Price"].mean()
total_purchase_gender=purchase_data.groupby(["Gender"])["Price"].sum()

#put 3 series toether 
purchase_analyses_gender=pd.concat([purchase_count_gender,ave_purchase_price_gender,total_purchase_gender], axis=1)
#name columns
purchase_analyses_gender.columns=["Purchase_count","ave_purchase_price", "total"]         
                                
                          
                                    
#merge with gender -demograohics to het the columns of total counf for each gender
n=pd.merge(purchase_analyses_gender,gender_demographics,on=["Gender"])
#calculate total purchase per gender 
n["per_person"]=n["total"]/n["Total_Count"]
#remove unwanted column 
m=n.iloc[:,[0,1,2,3,6]]
m

                                       


,Gender,Purchase_count,ave_purchase_price,total,per_person
0,Female,113,3.203009,361.94,4.468395
1,Male,652,3.017853,1967.64,4.065372
2,Other / Non-Disclosed,15,3.346000,50.19,4.562727


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
#define Bins and label for bins
age_bin=[0,9,14,19,24,29,34,39,100]
age_labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39",">40"]

#find all not duplicate people with their age , no other column
refined_age=purchase_data.drop_duplicates(subset=["SN","Age"],keep='first',inplace=False)
refined_age_only=refined_age.loc[:,["SN","Age"]]



In [11]:
#add categoical age columnd based on bins and label
refined_age_only["Age_Category"]=pd.cut(refined_age_only["Age"],bins=age_bin,labels=age_labels)

#same as gender calculation
age_demographics=pd.DataFrame(refined_age_only.groupby("Age_Category")["Age"].count())
age_demographics["Percent"]=age_demographics["Age"]/total_number_player*100
age_demographics.columns=["Total_count","Percentage_of_player"]
age_demographics

,Total_count,Percentage_of_player
Age_Category,,
<10,17,2.951389
10-14,22,3.819444
15-19,107,18.576389
20-24,258,44.791667
25-29,77,13.368056
30-34,52,9.027778
35-39,31,5.381944
>40,12,2.083333


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [12]:
#add age category column to main dat frame
purchase_data["Age_Category"]=pd.cut(purchase_data["Age"],bins=age_bin,labels=age_labels)

#do 3 calculation
purchase_count_age=purchase_data.groupby(["Age_Category"])["Item ID"].count()
ave_purchase_price_age=purchase_data.groupby(["Age_Category"])["Price"].mean()
total_purchase_age=purchase_data.groupby(["Age_Category"])["Price"].sum()

#put all in one data frame
purchase_analyses_age=pd.concat([purchase_count_age,ave_purchase_price_age,total_purchase_age,
                                    age_demographics["Total_count"]], axis=1)
#name the columns
purchase_analyses_age.columns=["Purchase_count","ave-purchase_price", "total_purchase","Total_count"]

#calculaye last column required 
purchase_analyses_age["Avg Total Purchase per Person"]=purchase_analyses_age["total_purchase"]/purchase_analyses_age["Total_count"]

#just show what they ask
purchase_analyses_age.iloc[:,[0,1,2,4]]

                                   
                                
                                    
                                    




,Purchase_count,ave-purchase_price,total_purchase,Avg Total Purchase per Person
Age_Category,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
>40,13,2.941538,38.24,3.186667


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
#calculate columns
purchase_count_sn=purchase_data.groupby(["SN"])["Item ID"].count()
ave_purchase_price_sn=purchase_data.groupby(["SN"])["Price"].mean()
total_purchase_sn=purchase_data.groupby(["SN"])["Price"].sum()

#put them togetther and rename 
spender=pd.concat([purchase_count_sn,ave_purchase_price_sn,total_purchase_sn],axis=1)
spender.columns=["Purchase_count","ave_price", "total_buy"]
spender.head()




,Purchase_count,ave_price,total_buy
SN,,,
Adairialis76,1,2.28,2.28
Adastirin33,1,4.48,4.48
Aeda94,1,4.91,4.91
Aela59,1,4.32,4.32
Aelaria33,1,1.79,1.79


In [14]:
sorted_spender=spender.sort_values("total_buy",ascending=False)
sorted_spender.head()

,Purchase_count,ave_price,total_buy
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [15]:
#same  way as above , this time fir item
purchase_group=purchase_data.groupby(["Item ID","Item Name"])
a=purchase_group["Purchase ID"].count()
b=purchase_group["Price"].sum()
c=purchase_group["Price"].max()
d=pd.concat([a,b,c],axis=1)
d.columns=["Purchase count","Total_buy","unit Price"]

#sort by purchase count
e=d.sort_values("Purchase count",ascending=False)
e.head()


,,Purchase count,Total_buy,unit Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,50.76,4.23
145,Fiery Glass Crusader,9,41.22,4.58
108,"Extraction, Quickblade Of Trembling Hands",9,31.77,3.53
82,Nirvana,9,44.10,4.90
19,"Pursuit, Cudgel of Necromancy",8,8.16,1.02


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [16]:
#sort by total purchase 
f=d.sort_values("Total_buy",ascending=False)
f.head()

,,Purchase count,Total_buy,unit Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,50.76,4.23
82,Nirvana,9,44.10,4.90
145,Fiery Glass Crusader,9,41.22,4.58
92,Final Critic,8,39.04,4.88
103,Singed Scalpel,8,34.80,4.35


#3 written description :
    * about 85% of the buyer are male .mich more than female and  others.Also males has more purchase count (652/780)
    * about 45% of the buyer are betwen 20-24 years old .Also this age goup has more purchase count than others.(365/780)
    *about 75%  of the buyer are between 15-29 years old.
    * There is not too much difeence in unit price and average purchase per person for different Gender and  also diffrent age group.
    *
    
    